In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt


## Spatial distribution of wildire occurrence in CA. 

In [3]:
# Read the Parquet file
data = pd.read_parquet("usgs-mtbs.parquet")

In [4]:
data

,Event_ID,irwinID,Incid_Name,Incid_Type,Map_ID,Map_Prog,Asmnt_Type,BurnBndAc,BurnBndLat,BurnBndLon,...,Perim_ID,dNBR_offst,dNBR_stdDv,NoData_T,IncGreen_T,Low_T,Mod_T,High_T,Comment,geometry
0,AK6448214467319840607,None,GLACIER CREEK,Wildfire,4534,MTBS,Initial (SS),1897,64.482,-144.673,...,506601519860705,-9999,-9999,9999,9999,550,165,-20,None,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x07\x01...
1,SD4308410285019840906,None,SANDOZ,Wildfire,4535,MTBS,Initial,2691,43.084,-102.850,...,None,211,-9999,-970,-150,220,700,9999,None,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00:\x01\x0...
2,MT4880411345819840819,None,NAPI PEAK,Wildfire,4536,MTBS,Extended,3680,48.804,-113.458,...,None,-62,-9999,-970,-150,160,308,515,None,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x90\x01...
3,MT4533610695819840826,None,KIRBY,Wildfire,4537,MTBS,Extended,1597,45.336,-106.958,...,None,52,-9999,-970,-150,150,309,525,None,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00E\x01\x0...
4,MT4561710632319840918,None,LAKE CREEK,Wildfire,4538,MTBS,Initial,3617,45.617,-106.323,...,None,-104,-9999,-970,-150,-50,175,9999,None,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00C\x02\x0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30919,FL2587108124920220306,None,OCHOPEE RX UNIT C (DR311),Prescribed Fire,10024901,MTBS,Initial,2500,25.868,-81.257,...,None,-33,27,-970,-150,0,9999,9999,None,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00]\x02\x0...
30920,FL3027208471520220201,None,UNNAMED,Prescribed Fire,10024910,MTBS,Initial,1071,30.266,-84.701,...,None,48,38,-970,-150,100,350,9999,None,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\xcc\x03...
30921,FL3043308448820220301,None,UNNAMED,Prescribed Fire,10024914,MTBS,Initial,2106,30.43,-84.491,...,None,34,22,-970,-150,80,400,9999,Burn scar to the east is under a different ID.,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\xb0\x01...
30922,AL3111908654520220504,EDF9CE76-7A31-4BA4-BD95-5DB4BA8605BE,CN BLUE SPRINGS EAST 17,Prescribed Fire,10024915,MTBS,Initial,1887,31.124,-86.547,...,None,-68,26,-970,-150,-53,485,9999,None,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\xbd\x00...


In [6]:
# Read the Parquet file metadata
parquet_file = pq.ParquetFile("usgs-mtbs.parquet")
all_columns = parquet_file.schema.names

# Exclude the problematic column
columns_to_load = [col for col in all_columns if col != 'RX_CONSUM']

# Load the file without 'RX_CONSUM'
s = pd.read_parquet("usgs-mtbs.parquet", columns=columns_to_load)
s.head()

,Event_ID,irwinID,Incid_Name,Incid_Type,Map_ID,Map_Prog,Asmnt_Type,BurnBndAc,BurnBndLat,BurnBndLon,...,Perim_ID,dNBR_offst,dNBR_stdDv,NoData_T,IncGreen_T,Low_T,Mod_T,High_T,Comment,geometry
0,AK6448214467319840607,None,GLACIER CREEK,Wildfire,4534,MTBS,Initial (SS),1897,64.482,-144.673,...,506601519860705,-9999,-9999,9999,9999,550,165,-20,None,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x07\x01...
1,SD4308410285019840906,None,SANDOZ,Wildfire,4535,MTBS,Initial,2691,43.084,-102.850,...,None,211,-9999,-970,-150,220,700,9999,None,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00:\x01\x0...
2,MT4880411345819840819,None,NAPI PEAK,Wildfire,4536,MTBS,Extended,3680,48.804,-113.458,...,None,-62,-9999,-970,-150,160,308,515,None,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x90\x01...
3,MT4533610695819840826,None,KIRBY,Wildfire,4537,MTBS,Extended,1597,45.336,-106.958,...,None,52,-9999,-970,-150,150,309,525,None,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00E\x01\x0...
4,MT4561710632319840918,None,LAKE CREEK,Wildfire,4538,MTBS,Initial,3617,45.617,-106.323,...,None,-104,-9999,-970,-150,-50,175,9999,None,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00C\x02\x0...


In [1]:
import os
import dask_geopandas as dg
import geopandas as gpd
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError

In [2]:

def get_s3_keys(bucket_name, prefix, client):
    """
    Fetches all the S3 keys associated with a specified prefix.

    Inputs:
    --------
    bucket_name : string
        The name of the S3 bucket.
    prefix : string
        The prefix to filter the keys (e.g., folder path).
    client : boto3 client object
        An S3 client returned by boto3.client.

    Returns:
    --------
    keys : list
        List of all keys that match the given prefix.
    """
    keys = []

    try:
        response = client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
        while response.get('Contents'):
            for obj in response['Contents']:
                keys.append(obj['Key'])

            if response['IsTruncated']:
                continuation_token = response['NextContinuationToken']
                response = client.list_objects_v2(Bucket=bucket_name, Prefix=prefix, ContinuationToken=continuation_token)
            else:
                break

    except (NoCredentialsError, PartialCredentialsError) as e:
        print(f"Error: {e}")
    
    return keys

In [3]:

def download_s3_file(bucket_name, key, local_dir, client):
    """
    Downloads a file from S3 if it doesn't already exist locally.

    Inputs:
    --------
    bucket_name : string
        The name of the S3 bucket.
    key : string
        The S3 key to download.
    local_dir : string
        The local directory to save the file.
    client : boto3 client object
        An S3 client returned by boto3.client.
    """
    local_fname = os.path.join(local_dir, os.path.basename(key))
    if not os.path.exists(local_fname):
        print(f"File not found locally. Downloading {key} from S3...")
        try:
            client.download_file(Bucket=bucket_name, Key=key, Filename=local_fname)
            print(f"Download complete: {local_fname}")
        except Exception as e:
            print(f"An error occurred during download: {e}")
    else:
        print(f"File already exists locally: {local_fname}")

In [4]:

def get_wildfire_boundaries(bucket_name, prefix, s3_client, local_dir="./data"):
    """
    Downloads wildfire boundary polygons from S3 and reads them into a Dask GeoDataFrame.

    Inputs:
    --------
    bucket_name : string
        The name of the S3 bucket.
    prefix : string
        The prefix to filter the wildfire boundary files.
    s3_client : boto3 client object
        An S3 client returned by boto3.client.
    local_dir : string, optional
        The local directory to store downloaded files (default is './data').

    Returns:
    --------
    wildfire_ddf : dask_geopandas.GeoDataFrame
        A Dask GeoDataFrame containing the wildfire boundary polygons.
    """
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)

    keys = get_s3_keys(bucket_name, prefix, s3_client)
    if not keys:
        print("No files found in the specified S3 bucket/prefix.")
        return None

    for key in keys:
        download_s3_file(bucket_name, key, local_dir, s3_client)

    # Load all downloaded Parquet files into a Dask GeoDataFrame
    wildfire_ddf = dg.read_parquet(os.path.join(local_dir, "*.parquet"), gather_spatial_partitions=False)
    return wildfire_ddf

In [5]:

# Example usage
if __name__ == "__main__":
    # Initialize S3 client
    s3_client = boto3.client('s3')

    # S3 bucket and prefix details
    bucket_name = "your-s3-bucket-name"  # Replace with the actual S3 bucket name
    prefix = "wildfire-boundary-polygons/"  # Replace with the actual prefix for wildfire polygons

    # Fetch and process the wildfire boundaries
    wildfire_ddf = get_wildfire_boundaries(bucket_name, prefix, s3_client)

    if wildfire_ddf is not None:
        print(wildfire_ddf.head())

Error: Unable to locate credentials
No files found in the specified S3 bucket/prefix.
